<h1 align="center">Regression with NumPy</h1>

<h2 align="center">Assignment 1 - Regression</h2>

You are tasked to develop a regression model that best fits the data below. You will turn in a Jupyter notebook in PDF format that describes your method, and provides and demonstrates your code.

All data sets are curated from the UCI Machine Learning repository.

- Airfoil Self-Noise: https://archive.ics.uci.edu/ml/datasets/Airfoil+Self-Noise 
- Yacht Hydrodynamics: https://archive.ics.uci.edu/ml/datasets/Yacht+Hydrodynamics
- Concrete Slump: https://archive.ics.uci.edu/ml/datasets/Concrete+Slump+Test

## Importing packages

In [25]:
# Import Pandas
import pandas as pd

# Import Numpy
import numpy as np

# Import Random
import random

# Scipy Stats
from scipy import stats

# K FOld
from sklearn.cross_validation import KFold

# Import matplotlib pyplot
from matplotlib import pyplot as plt

# Import Bokeh
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import d3
from bokeh.io import output_notebook
from bokeh.models import Legend
output_notebook()

# Import warnings
import warnings 
warnings.filterwarnings('ignore')

Loading BokehJS ...

## Reading the data from the web

Let us first read the data from the web and store those into pandas dataframe for further processing.

In [2]:
# Airfoil Self-Noise data
airfoil_df = pd.read_table(
    filepath_or_buffer = "https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat",
    names = ["Frequency", "Angle of attack", "Chord length", "Free-stream velocity", 
             "Suction side displacement", "Scaled sound pressure"])

# Yacht Hydrodynamics
yacht_hydro_df = pd.read_table(
    filepath_or_buffer = "https://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data",
    names = ["Longitudinal buoyancy", "Prismatic coefficient", "Length-displacement", "Beam-draught", 
             "Length-beam", "Froude number", "Residuary resistance"], delim_whitespace = True)

# Concrete Slump
concrete_slump_df = pd.read_csv(
    filepath_or_buffer = "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data",
    names = ["Cement", "Slag", "Fly ash", "Water", "SP", "Coarse Aggr.", "Fine Aggr.", "SLUMP", "FLOW",
             "Compressive Strength"], skiprows = 1)

As we have downloaded and imported those as pandas dataframe format here, lets verify that we have all the three dataset by viewing the top 5 rows in the dataset.

In [3]:
airfoil_df.head()

,Frequency,Angle of attack,Chord length,Free-stream velocity,Suction side displacement,Scaled sound pressure
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [4]:
yacht_hydro_df.head()

,Longitudinal buoyancy,Prismatic coefficient,Length-displacement,Beam-draught,Length-beam,Froude number,Residuary resistance
0,-2.3,0.568,4.78,3.99,3.17,0.125,0.11
1,-2.3,0.568,4.78,3.99,3.17,0.150,0.27
2,-2.3,0.568,4.78,3.99,3.17,0.175,0.47
3,-2.3,0.568,4.78,3.99,3.17,0.200,0.78
4,-2.3,0.568,4.78,3.99,3.17,0.225,1.18


In [5]:
concrete_slump_df.head()

,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP,FLOW,Compressive Strength
1,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0,62.0,34.99
2,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0,20.0,41.14
3,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0,20.0,41.81
4,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0,21.5,42.08
5,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0,64.0,26.82


## Question 1

Develop a function with the following first line:

def my_regression(trainX, testX, noutputs)

The input variables are:

- trainX - an [ntrain x (nfeature + noutputs)] array that contains the features in the first 'nfeature' columns and the outputs in the last 'noutput' columns
- testX - an [ntest x nfeature] array of test data for which the predictions are made
- noutputs - the number of output columns in trainX

The output should be an [ntest x noutputs] array, which contains the prediction values for the testX data.
Your my_regression code should do some kind of cross-validation to determine the right model for the training data, e.g., linear vs. polynomial vs. radial basis functions. Then this model is applied to the test data to make a prediction.

### Rules:

- Your code should not use any regression library, but can use NumPy as this will be useful for array handling.
- No other inputs are allowed.
- You may not use the testX data to train your model. Notice that outputs are not passed in for testing data.
- Your code can perform data scaling (scaling features to the interval [0,1] or z-score scaling), can use any basis functions you want, cross-validation on the training data, regularization, and model selection.

Present your findings for each data set as a Jupyter notebook. Submit as a PDF. Present your squared error for several cross-validation fold of each data set.

## Solution

I am dividing the whole project into three components which are as follows

- Data Processing
- Regression 
- Regression Plots

Each components listed above will be coded as seperate python classes and can be reused. The doc string are written for each classes and functions within the classes. Although, I am just setting the seed value for reproduciable research purpose.

In [6]:
# Setting the seed value
seed = 123
import random
random.seed(seed)
np.random.seed(seed)

### Data Processing

In [7]:
class DataProcessing:
    """Data Processing opertation 
    
    Processing of data such as Normalization, spliting the data,
    folding the data can be performed on the pandas dataframe
    
    Arguments:
        Pandas dataframe that need to processed
    """
    
    def __init__(self):
        """Variable Initialization
        
        Initialize the class variables which can be accessed using 
        the self or class objects
        
        Arguments:
            None
        Returns:
             None
        """
        pass
    
    def z_score_norm(self, data):
        """Normalize the data using z-score
        
        Normalize the data which is mostly a pandas dataframe and 
        outputs normalized dataframe of the same using z-score 
        normalization.https://en.wikipedia.org/wiki/Standard_score
        
        Arguments:
            data: Pandas Dataframe to be normalized
            
        Returns:
            Normalized Pandas Dataframe
        """
        # Finding the z-score using mean and standard deviation of each column
        # Broadcasting is applied over the data
        normalized_df = (data - data.mean()) / data.std()
        return pd.DataFrame(normalized_df)
    
    def train_test_split(self, data, cut_at = 0.75):
        """ Data is split into train and test set
        
        Split the data set into two random chuncks of train and test set 
        using the cut_at value. The cut_at value literally means how much 
        percentage of data you want to be in the training set and rest
        of those goes to the testing set.
        
        Arguments:
            data: Pandas Dataframe to be split into train and test
            cut_at: Threshold for what percentage of data needs top be 
                in the training set. Ranges from 0 to 1. Default to 0.75
                
        Returns:
            train: Training set of the data
            test: Testing set of the data
        """
        # Random sample of the train set is acquired
        train = data.sample(frac = cut_at, random_state = 200)
        # Rest are put in the test set
        test = data.drop(train.index)
        return train, test
    
    def pred_target_split(self, data, noutputs = 1):
        """ Split the Predictors and output variables
        
        The Input data variable is take and predictors and n-output vectors 
        are drawn out of the complete data. Note that this function works only
        when the output varibles are end/last column of the data.
        
        Arguments:
            data: Pandas Dataframe to be split into Predictors and target
            noutputs: Number of output variables in the dataset. Default to 1
            
        Returns:
            predictors: Predictors of the data
            outputs: Output variable of the data
        """
        # Subsetting the predictors 
        predictors = data.iloc[:, 0:-noutputs]
        # Subsetting the outputs
        outputs = data.iloc[:, -noutputs:]
        return predictors, outputs
    
    def k_fold(self, data, k = 7):
        """ Create K folds of the data
        
        Data is split into K number of chunks depending on the k argument of 
        the function that can be used up for the cross validation purpose.
        
        Arguments:
            data: Pandas Dataframe to be split into K Folds
            k: Number of folds. Default to 7
            
        Returns:
            index: Row index for each train and test set of the folds
        """
        # Creating K-Fold cross validation index.
        index = KFold(len(data), n_folds = k)
        return index

### Regression

In [8]:
class Regression:
    """Perform regression on data
    
    Looks over the predictor and target outputs with linear and Gaussian
    basis function with different lamda values and doing the model selection
    based on k-fold cross validation on the metrics of least mean sum of 
    squares error value.
    
    Arguments:
        k_fold: Number of folds of the data used for the model selection
        lamda_values: List of float values generally of increasing log scale
            that are used in the closed solution to find best optimal lamda 
            for the final model.
    """
    
    # Creating the data Processing object for training data
    data_process = DataProcessing()
    
    def __init__(self, k_fold = 7):
        self.k_fold = k_fold
        self.lamda_values = [0, 0.0001, 0.001, 0.01, 0.1, 0, 1, 10, 100]
        self.kernels = ["linear", "gaussian"]
        self.parameters_list = []
        self.n_points = 10
        self.model_param = []
        
    def lmse(self, actual, pred):
        """ Find the Prediction LMSE
        
        LMSE -  Least Mean Square Error is the average squared difference 
        between the prediction and actual values. The inputs are actual and 
        predicted values that outputs the LMSE rounded off to two digits.
        
        Arguments:
            actual: Actual value from the dataset. continuous value.
            pred: Predicted value for the data. continuous value.
            
        Returns:
            lmse: Least Mean Square Error value
        """
        # Computing the LMSE and rounding of to two digits
        lmse = round(np.sum((actual - pred) ** 2) / len(actual), 2)
        return lmse

    def gaussian_kernel(self, data, n_points = 10):
        """ Apply gaussian function on the data 
        
        Dataset with x feature vector is passed over the n - gaussian function that
        is computed using random pick of observation from the dataset and converted 
        into n feauture vectors from x features. This makes the data to be around your
        n data points you selected.
        
        Arguments:
            data: Data to be changed into gaussian form
            n_points: Number of gaussian kernels/functions/points to refine into
        
        Returns:
            gaussian: N feature vectors represetation of the input dataset.
        """
        # Set the sigma value to be no. of original features
        sigma = data.shape[1]
        # Setting the random seed and sampling the points from the data
        random.seed(100)
        points = random.sample(list(data.index), n_points)
        # Initialize the gaussian kernel feature vector
        gaussian = np.zeros((data.shape[0], n_points))
        col = 0
        # Iterate through the n points to come up with n gaussian feauture vectors
        for point in points:
            gaussian[:, col] = np.exp(-np.linalg.norm(data - data.loc[point, :], 2, axis = 1) ** 2 
                                      / (2. * sigma ** 2))
            col += 1
        return gaussian
    
    def cv_fit(self, x_train, y_train, k_fold):
        """ Model Selection using Cross validation
        
        Selecting the best fit regression model and hyper parameters such 
        as lamda and basis functions using cross validation on the feauture 
        vectors and our output variable(s) and linear regression's closed
        form solution.
        
        Arguments:
            x_train: Predictors of the Training set
            y_train: Output of the Training set
            k_fold: Number of fold for the cross validation
            
        Returns:
            model_param: A dictinoary consists of selected model weights, 
                lamda value, lmse value, and kernel functions
        """
        
        # Initialize variables 
        weights = np.zeros((x_train.shape[1],))
        best_weights = weights
        identity = np.identity(x_train.shape[1])
        best_lmse = float("inf")
        best_lamda = 0.
        best_kernel = ""
        
        # K Fold cross validation index generetion
        cv = self.data_process.k_fold(data = x_train, k = k_fold)
        
        # Looping through the basis function
        for kernel in self.kernels:
            # Chacking the Gaussian function
            if kernel is "gaussian":
                x_train = self.gaussian_kernel(pd.DataFrame(x_train), self.n_points)
                identity = np.identity(x_train.shape[1])
            # Looping through Lamda
            for lamda in self.lamda_values:
                lmse = 0
                for train_cv, test_cv in cv:
                    # Split up the train and test for each iteration
                    x_train_cv = x_train[train_cv]
                    y_train_cv = y_train[train_cv]
                    x_test_cv = x_train[test_cv]
                    y_test_cv = y_train[test_cv]
                    # Closed form solution for each lamda values
                    inverse = np.linalg.inv(lamda*identity + np.dot(x_train_cv.T, x_train_cv))
                    data_term = np.dot(x_train_cv.T, y_train_cv)
                    weights = np.dot(inverse, data_term)
                    # Predict with the test set
                    prediction = np.dot(x_test_cv, weights)
                    # Find the LMSE value for the set
                    lmse += (np.sum((y_test_cv - prediction) ** 2) / len(y_test_cv))
                # LMSE for each lamda
                lmse = lmse / k_fold
                # Preparing the parameter list for further analysis
                # Storing the result in class variable
                parameters = {'basis_function': kernel, 
                              'lamda': lamda,
                              'lmse': lmse,
                              'weights': weights}
                self.parameters_list.append(parameters)
                # Update the best parameters
                if best_lmse > lmse:
                    best_lmse = lmse
                    best_lamda = lamda
                    best_weights = weights
                    best_kernel = kernel 
        model_param = {"weights": best_weights, "kernel": best_kernel, "lamda": best_lamda, "lmse": best_lmse}
        self.model_param.append(model_param)
        return model_param

    def my_regression(self, trainX, testX, noutputs):
        """ Make Prediction for continuous output(s)
        
        Takes training set and performs cross validation to select the best 
        parameter (lamda value of the closed form solution, basis function) 
        for the model selection and use that model on the testing set to 
        output the prediction for the testing data.
        
        Arguments:
            trainX - an [ntrain x (nfeature + noutputs)] array that contains
                the features in the first 'nfeature' columns and the outputs 
                in the last 'noutput' columns
            testX - an [ntest x nfeature] array of test data for which the 
                predictions are made
            noutputs - the number of output columns in trainX
            
        Returns:
            prediction: [ntest x noutputs] array, which contains the prediction 
                values for the testX data
        """        
        # Normalize the train and test data
        trainX.iloc[:, 0:-noutputs] = self.data_process.z_score_norm(trainX.iloc[:, 0:-noutputs])
        testX = self.data_process.z_score_norm(testX)        
        # Split the train data into predictors and target variables
        x_train, y_train = self.data_process.pred_target_split(trainX, noutputs = noutputs)              
        # Include the bias term in the feature vector at the last
        x_train["bias"] = 1
        testX["bias"] = 1
        # Convert Dataframe to Numpy array
        x_train = x_train.values
        y_train = y_train.values 
        # Intializing the prediction array with zeros
        prediction = np.zeros((testX.shape[0], noutputs)) 
        # Get the best fit for the data with cross validation
        model = self.cv_fit(x_train, y_train, self.k_fold)
        # Check if the kernel is gaussian
        if model["kernel"] is "gaussian":
            # Redefine in gaussian feature
            gaus = self.gaussian_kernel(pd.DataFrame(np.concatenate((x_train, testX.values))),
                                                self.n_points)
            x_train_gaus = gaus[0:len(x_train), :]  
            x_test_gaus = gaus[len(x_train):, :]
            # Initialize the identity matrix
            identity = np.identity(x_train_gaus.shape[1])
            # Train with the whole training data
            inverse = np.linalg.inv(model["lamda"] * identity + np.dot(x_train_gaus.T, x_train_gaus))
            data_term = np.dot(x_train_gaus.T, y_train)
            weights = np.dot(inverse, data_term)
            # Predict with the test data
            # Inverse/rescale for the z-score norm
            prediction = np.dot(x_test_gaus, weights)
        else:
            # Initialize the identity matrix
            identity = np.identity(x_train.shape[1])
            # Train with the whole training data
            inverse = np.linalg.inv(model["lamda"] * identity + np.dot(x_train.T, x_train))
            data_term = np.dot(x_train.T, y_train)
            weights = np.dot(inverse, data_term)
            # Predict with the test data
            # Inverse/rescale for the z-score norm
            prediction = np.dot(testX.values, weights)
        return prediction

Above we had our main two classes that contains the function needed for calculating the regression using numpy packages. Let's use those find the prediction for each dataset one by one.

## Airfoil dataset

In [9]:
airfoil_regressor = Regression()
airfoil_data_process = DataProcessing()
airfoil_df_train, airfoil_df_test = airfoil_data_process.train_test_split(airfoil_df)
prediction = airfoil_regressor.my_regression(airfoil_df_train, airfoil_df_test.iloc[:, 0:-1], 1)
prediction.reshape(len(prediction), )

array([ 127.33113782,  117.64481597,  126.01235471,  125.51458539,
        125.17825477,  113.40668309,  125.01623034,  123.60364173,
        123.81723075,  123.14456951,  126.35294204,  126.01661142,
        122.3169746 ,  116.93568469,  124.01837754,  123.48024855,
        123.21118406,  121.19320034,  120.31874073,  119.17521662,
        116.08097492,  113.79392671,  126.41941243,  126.28488018,
        125.88128344,  125.27588832,  124.26689647,  122.71977562,
        113.50431664,  123.25280345,  119.55316663,  117.06432005,
        122.4359302 ,  119.61075299,  114.49852757,  125.76947722,
        125.5945853 ,  125.09681598,  124.28962249,  121.06084855,
        112.98891367,  109.62560748,  122.83163405,  122.40785747,
        119.89210443,  119.01764482,  129.55299226,  129.08212939,
        128.5440004 ,  126.99687955,  125.85335544,  128.61387593,
        127.53761794,  126.99948895,  127.27286116,  127.04415634,
        125.42976937,  124.75710813,  121.39380193,  126.90507

Above we have the prediction values for the airfoil dataset using the selected model on the test set. Lets us see what are parameters used for the model using the model param variable of the regressor class object.

In [10]:
airfoil_regressor.model_param

[{'kernel': 'linear',
  'lamda': 0.1,
  'lmse': 23.665798767415858,
  'weights': array([[  -4.10692447],
         [  -2.40935473],
         [  -3.39686533],
         [   1.47280792],
         [  -2.06332953],
         [ 124.79044843]])}]

Thus we can see clearly that the best model for the airfoil dataset after 10 fold cross validation model selection has the following parameters and values

- Kernel : Linear
- Lamda value : 0.1
- Least Mean Square Error value: ~23.67

Lets us the see how the best model is performing over the test data we set apart at first itself using least mean square value.

In [11]:
lmse = airfoil_regressor.lmse(airfoil_df_test.iloc[:, -1].values.reshape(len(airfoil_df_test), 1), prediction)
print(lmse)

22.33


Thus we have Least Mean Square Error of 22.33 on the test set using Linear basis function itself and lamda as 0.1. It shows that the airfoil dataset have good result with the simple linear basis function itself instead of a complex Gaussian one. In addition, I have done small analysis over the cross validation data of this dataframe using some visualization to say why and how I selected linear to be my basis linear function and lamda value for the model. I have also created a plot function for reusing it for later times.

In [12]:
# Custom style attribute function
# Best practive for bokeh users
def model_param_analysis(data, title, width = 800, height = 600, xlab = "X-axis", ylab = "Y-axis", line_width = 4):
    # Creating the variables for the color prop
    colors_list = [d3['Category20b'][17][i] for i in [2, 6]]
    # Create a parameter dataframe from the regressor object
    df = pd.DataFrame(data)
    # Structuring the x and y axis values
    basis_function = df.basis_function.unique()
    xs = [df.lamda.unique()] * len(basis_function)
    ys = [df.loc[df["basis_function"] == x]["lmse"] for x in basis_function]
    # Create the figure object
    p = figure(width = width, height = height, title = title, x_axis_label = xlab, y_axis_label = ylab)
    # Iterate through each degree and draw the lines
    for (col_name, colr, x, y) in zip(basis_function, colors_list, xs, ys):
        current_plot = p.line(x, y, color = colr, legend = col_name.upper(), line_width = line_width)
        p.legend.location = "center"
    # Add plot style attributes
    p.title.text_font_size = "20pt"
    p.title.align = "center"
    p.xaxis.axis_label_text_font_size = "20pt"
    p.yaxis.axis_label_text_font_size = "20pt"
    return show(p)

# Show the plot
model_param_analysis(data = airfoil_regressor.parameters_list, title = "Airfoil Dataset", width = 800, height = 600, 
                         xlab = "Lamda", ylab = "Mean Square Error", line_width = 4)

From the above plot we can see clearly that, linear basis function is better performing for gaussian at all the lambda values for this airfoil data set. And the best lamda value we have selected for our final model is 0.1. You can see that of all points on the plot, least mean square error is at 0.1 approximately. And thus we selected linear basis with lamda value as 0.1 for our model.

## Yachat Hydro Dataset

In [13]:
yacht_hydro_regressor = Regression()
yacht_hydro_data_process = DataProcessing()
yacht_hydro_df_train, yacht_hydro_df_test = yacht_hydro_data_process.train_test_split(yacht_hydro_df)
prediction = yacht_hydro_regressor.my_regression(yacht_hydro_df_train, yacht_hydro_df_test.iloc[:, 0:-1], 1)
prediction.reshape(len(prediction))

array([ -8.03388298e+00,  -4.98197638e+00,   1.12183682e+00,
         1.02775566e+01,   1.33294632e+01,  -8.01210760e+00,
         2.86107716e+01,   1.92540344e+01,   2.23059410e+01,
        -8.48793339e+00,   1.89792260e+01,   2.20311326e+01,
         3.11868524e+01,  -4.12571466e+00,   1.11338183e+01,
         2.02895381e+01,   2.63933513e+01,   2.94452579e+01,
        -5.57093867e+00,   6.63668772e+00,   3.10519405e+01,
        -5.55399800e+00,   1.27574416e+01,  -7.34286100e+00,
        -1.23904780e+00,   1.81285880e+00,   7.91667199e+00,
         1.40204852e+01,   2.01242984e+01,   3.23319248e+01,
         5.56073525e-01,   2.49713263e+01,  -4.86829637e+00,
         4.28742343e+00,   7.33933003e+00,   1.34431432e+01,
        -9.17698823e+00,  -2.12684411e-02,   6.08254475e+00,
         9.13445135e+00,   2.13420777e+01,   2.74458909e+01,
        -4.32436723e+00,  -1.27246063e+00,   1.77944596e+00,
         4.83135256e+00,   7.88325916e+00,   1.39870724e+01,
         1.70389790e+01,

Above we have the prediction values for the yachat hydro dataset using the selected model on the test set. Lets us see what are parameters used for the model using the model param variable of the regressor class object.

In [14]:
yacht_hydro_regressor.model_param

[{'kernel': 'linear',
  'lamda': 10,
  'lmse': 85.289946954115365,
  'weights': array([[  0.52099247],
         [ -0.59127749],
         [  0.13064916],
         [ -0.56454935],
         [ -0.60913435],
         [ 12.22693704],
         [ 10.7282111 ]])}]

Thus we can see clearly that the best model for the yachat hydro dataset after 10 fold cross validation model selection has the following parameters and values

- Kernel : Linear
- Lamda value : 10
- Least Mean Square Error value: ~85.28

Lets us the see how the best model is performing over the test data we set apart at first itself using least mean square value.

In [15]:
lmse = yacht_hydro_regressor.lmse(yacht_hydro_df_test.iloc[:, -1].values.reshape(len(yacht_hydro_df_test), 1), prediction)
print(round(lmse, 2))

75.19


Thus we have Least Mean Square Error of 75.19 on the test set using Linear basis function itself and lamda as 10. It shows that the yachat dataset have good result with the simple linear basis function itself instead of a complex Gaussian one. In addition, I have done small analysis over the cross validation data of this dataframe using some visualization to say why and how I selected linear to be my basis linear function and lamda value for the model. I will be using the previously created custom plot function to do this.

In [16]:
# Show the plot
model_param_analysis(data = yacht_hydro_regressor.parameters_list, title = "Yachat Hydro Dataset", width = 800,  
                         height = 600, xlab = "Lamda", ylab = "Mean Square Error", line_width = 4)

This is interesting. If you see the plot you might decide that gaussian function might be having best value at lamda around 1. But if you zoom and see the plot linear basis function has much lower value at lamda = 10. Thus linear basis function is better performing for gaussian at lambda value equal to 10 for this yachat hydro data set. And the best lamda value we have selected for our final model is 10. And thus we selected linear basis with lamda value as 10 for our model.

## Concrete slump dataset

In [17]:
concrete_slump_regressor = Regression()
concrete_slump_data_process = DataProcessing()
concrete_slump_df_train, concrete_slump_df_test = concrete_slump_data_process.train_test_split(concrete_slump_df)
prediction = concrete_slump_regressor.my_regression(concrete_slump_df_train, concrete_slump_df_test.iloc[:, 0:-3], 3)
prediction

array([[ 13.20889869,  40.6643547 ,  41.45456363],
       [  9.47096006,  30.39801961,  38.82278299],
       [ 35.90407116,  92.67574756,  25.77788643],
       [ 23.08920266,  57.39856206,  29.78983861],
       [ 16.17209085,  40.38097971,  39.51512545],
       [ 14.13840173,  39.47591897,  36.33942465],
       [ 27.44532623,  70.48655395,  31.91084908],
       [ 16.70645531,  40.95980875,  21.17522819],
       [ 14.61037275,  39.74694377,  37.32890858],
       [ 20.94497296,  63.88409334,  44.68178252],
       [ 20.76697335,  58.97714942,  42.21273908],
       [ 17.77988688,  54.11335284,  38.11553217],
       [ 25.48940007,  71.55320205,  30.44791725],
       [ 24.21239932,  60.17300306,  17.47401764],
       [ 17.68396282,  46.35670145,  34.46348158],
       [ 22.31648071,  60.34223297,  31.31118298],
       [ 21.77312002,  63.16181578,  47.60858486],
       [ 23.45462449,  64.45497984,  42.56521536],
       [ 20.63679437,  50.5625387 ,  36.84569472],
       [ 23.91394107,  58.21924

Above we have the prediction values for the concrete dataset using the selected model on the test set. Each column of the array represents the prediction for each of the three output values of concrete slump dataset. Lets us see what are parameters used for the model using the model param variable of the regressor class object.

In [18]:
concrete_slump_regressor.model_param

[{'kernel': 'gaussian',
  'lamda': 0,
  'lmse': 204.17858171296729,
  'weights': array([[-1907.10615157, -4886.679646  ,  2199.20283645],
         [-1575.26809789, -1543.28770302,  1067.64196235],
         [ -481.90347933,  -162.94041964,   531.0106334 ],
         [-1350.63734208, -3234.77550261,  1739.3611437 ],
         [ 2150.15656111,  4716.38228893, -2452.31684967],
         [ -553.4238615 ,  -797.41533925,   368.45154025],
         [ 1816.91376834,  4082.41909882, -1849.58460534],
         [ 2318.76131361,  2174.31334234, -1722.13423763],
         [  559.4798499 ,   396.58533667,  -409.75827288],
         [-1052.43066878,  -826.29232017,   638.7306044 ]])}]

This dataset is somewhat different from the previous two. Since it had three output values the Least mean square error is calculated as a average over all the three outputs while during the model selection. By doing that over 10 fold cross validation, we have the following bst model parameters

- Kernel : gaussian
- Lamda value : 0
- Least Mean Square Error value: ~204.18

Lets us the see how the best model is performing over the test data we set apart at first itself using least mean square value.

In [19]:
lmse_list = []
for i in range(0, 3):
    lmse = concrete_slump_regressor.lmse(concrete_slump_df_test.iloc[:, -3 + i].values.reshape(len(concrete_slump_df_test), ), 
                          prediction[:, i].reshape(len(prediction), ))
    lmse_list.append(round(lmse, 2))

print("Least mean square errors:")
print("SLUMP:", lmse_list[0])
print("FLOW:", lmse_list[1])
print("Compressive Strength:", lmse_list[2])

Least mean square errors:
SLUMP: 82.42
FLOW: 321.81
Compressive Strength: 14.72


Thus we have above the Least Mean Square Error on the test set using Gaussian basis function and lamda as 0. In addition, I have done small analysis over the cross validation data of this dataframe using some visualization to say why and how I selected Gaussian to be my basis linear function and lamda value for the model. I will be using the previously created custom plot function to do this. Remeber that the least mean square error is average over all the indepedent variables in the dataset.

In [20]:
# Show the plot
model_param_analysis(data = concrete_slump_regressor.parameters_list, title = "Concrete slump Dataset", width = 800,  
                         height = 600, xlab = "Lamda", ylab = "Mean Square Error", line_width = 4)

From the plot it is so obvious that the gaussian is out performing the linear basis function with the concrete slump dataset. We can see that the gaussian is reaching two lower points related to linear basis function at 0 and 1. 

When I was checking the lmse value of training the whole set with linear basis function and lamda as 0 the lmse value was bit lower compared to the above lmse got using gaussian model. However, considering the real world scanario it is always better to go with model parameters which we got using cross validation.

Thus the final best model I have selected was with Gaussian basis function and lamda value as 0.

## Proof testing mode

In order to just do double chack the lmse value i got from my regression function, I used sklearn regression to see how close i am getting results in regards to sklearn.

In [21]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

def sklearn_regression(trainX, testX, noutputs = 1):
    # Create linear regression object
    regr = linear_model.LinearRegression()
    
    # Create DataProcessing class object 
    data_process = DataProcessing()
    
    x_train, y_train = data_process.pred_target_split(trainX, noutputs = noutputs)

    # Train the model using the training sets
    regr.fit(x_train, y_train)

    # Make predictions using the testing set
    pred = regr.predict(testX.iloc[:, 0:-noutputs].values)

    print("Mean squared error: %.2f"
          % mean_squared_error(testX.iloc[:, -noutputs].values, pred))

## Airfoil dataset

In [22]:
airfoil_df_train, airfoil_df_test = airfoil_data_process.train_test_split(airfoil_df)
sklearn_regression(airfoil_df_train, airfoil_df_test)

Mean squared error: 22.36


I got very close value to this for my regression function.

## Yachat hydro dataset

In [23]:
yacht_hydro_df_train, yacht_hydro_df_test = yacht_hydro_data_process.train_test_split(yacht_hydro_df)
sklearn_regression(yacht_hydro_df_train, yacht_hydro_df_test)

Mean squared error: 73.66


I got very close value to this for my regression function.

## Concrete slump dataset

In [24]:
concrete_slump_df_train, concrete_slump_df_test = concrete_slump_data_process.train_test_split(concrete_slump_df)

# Create linear regression object
regr = linear_model.LinearRegression()

x_train, y_train = concrete_slump_data_process.pred_target_split(concrete_slump_df_train, noutputs = 3)

for i in range(0, 3):
    # Train the model using the training sets
    regr.fit(x_train, y_train.iloc[:, i])

    # Make predictions using the testing set
    pred = regr.predict(concrete_slump_df_test.iloc[:, 0:-3].values)

    print("Mean squared error: %.2f"
          % mean_squared_error(concrete_slump_df_test.iloc[:, i-3].values, pred))

Mean squared error: 74.01
Mean squared error: 217.23
Mean squared error: 8.35


I got very close value to this for my regression function.

### Rubric:

- (30 points) - The regression works and you produce some kind of result.
- (30 points) - Are the results reasonable? Do you get squared errors that are close to mine? (I'll release linear basis function sqaured errors)
- (20 points) - Can you do better than linear basis function prediction?
- (20 points) - Is your code commented? Is your analysis strong? Do your plots look nice? Does your overall document look nice?